## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Dataset Exploration</a>  
2. <a href="#Build-the-baseline-model">Build the baseline model</a>  
3. <a href="#Train-and-Test-the-Network--(PART-A)">Part A</a>  
4. <a href="#Train-and-Test-the-Network--(PART-B)">Part B</a>  
5. <a href="#Train-and-Test-the-Network--(PART-C)">Part C</a>
6. <a href="#Train-and-Test-the-Network--(PART-D)">Part D</a>  

</font>
</div>

<a id="item31"></a>

## I. Dataset Exploration 

In [1]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std

In [2]:
# Download the data and read it into a <em>pandas</em> dataframe.
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
#check how many data points we have.
concrete_data.shape

(1030, 9)

In [4]:
#Let's check the dataset for any missing values.
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

In [6]:
#Split data into predictors and target. 
#The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
#Also obtain normalized data by substracting the mean and dividing by the standard deviation.
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
#Save the number of predictors to *n_cols*
n_cols = predictors.shape[1] 

## Build the baseline model

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense

# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
    return model

Using TensorFlow backend.


## Train and Test the Network  (PART A)

In [12]:
#PART A
mean_squared_error_list=[]

# Build the model
model = regression_model()

for i in range (0,50):    
    print("Iteration #",i)
    # Train and test the model at the same time using the fit method. 30% of the data for validation.
    history = model.fit(predictors, target, validation_split=0.3, epochs=50, verbose=2)
    mean_squared_error_list.append(history.history['mean_squared_error'][len(history.history['mean_squared_error'])-1]) # Last MSE reading

Iteration # 0
Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 7331.7650 - mean_squared_error: 7331.7650 - val_loss: 2019.9730 - val_mean_squared_error: 2019.9730
Epoch 2/50
 - 0s - loss: 2854.3176 - mean_squared_error: 2854.3176 - val_loss: 1659.0219 - val_mean_squared_error: 1659.0219
Epoch 3/50
 - 0s - loss: 2296.5379 - mean_squared_error: 2296.5379 - val_loss: 1450.4708 - val_mean_squared_error: 1450.4708
Epoch 4/50
 - 0s - loss: 1992.3395 - mean_squared_error: 1992.3395 - val_loss: 1252.4742 - val_mean_squared_error: 1252.4742
Epoch 5/50
 - 0s - loss: 1720.5552 - mean_squared_error: 1720.5552 - val_loss: 1084.7570 - val_mean_squared_error: 1084.7570
Epoch 6/50
 - 0s - loss: 1503.7626 - mean_squared_error: 1503.7626 - val_loss: 944.6924 - val_mean_squared_error: 944.6924
Epoch 7/50
 - 0s - loss: 1300.8909 - mean_squared_error: 1300.8909 - val_loss: 831.3895 - val_mean_squared_error: 831.3895
Epoch 8/50
 - 0s - loss: 1146.2880 - mean_squared_error: 1146.2880 - 

In [13]:
print("Part A")
print("Mean of the mean squared errors: " , mean(mean_squared_error_list))
print("Standard deviation of the mean squared errors: ", std(mean_squared_error_list))

Part A
Mean of the mean squared errors:  120.77343989341831
Standard deviation of the mean squared errors:  22.403923100670788


## Train and Test the Network  (PART B)

In [14]:
#PART B
mean_squared_error_list=[] # reset the list for use again in new training and testing( Part B)

# Build the model
model = regression_model()
    
for i in range (0,50):
    # Train and test the model at the same time using the fit method. 30% of the data for validation.
    history = model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2)
    mean_squared_error_list.append(history.history['mean_squared_error'][len(history.history['mean_squared_error'])-1])



Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 1740.8785 - mean_squared_error: 1740.8785 - val_loss: 1247.9975 - val_mean_squared_error: 1247.9975
Epoch 2/50
 - 0s - loss: 1722.3742 - mean_squared_error: 1722.3742 - val_loss: 1235.9390 - val_mean_squared_error: 1235.9390
Epoch 3/50
 - 0s - loss: 1704.5001 - mean_squared_error: 1704.5001 - val_loss: 1224.3534 - val_mean_squared_error: 1224.3534
Epoch 4/50
 - 0s - loss: 1687.1941 - mean_squared_error: 1687.1941 - val_loss: 1213.0223 - val_mean_squared_error: 1213.0223
Epoch 5/50
 - 0s - loss: 1670.4378 - mean_squared_error: 1670.4378 - val_loss: 1202.1614 - val_mean_squared_error: 1202.1614
Epoch 6/50
 - 0s - loss: 1653.7192 - mean_squared_error: 1653.7192 - val_loss: 1191.7707 - val_mean_squared_error: 1191.7707
Epoch 7/50
 - 0s - loss: 1636.7594 - mean_squared_error: 1636.7594 - val_loss: 1181.6227 - val_mean_squared_error: 1181.6227
Epoch 8/50
 - 0s - loss: 1619.9437 - mean_squared_error: 1619.9437 - val_loss: 

In [15]:
print("Part B")
print("Mean of the mean squared errors: " , mean(mean_squared_error_list))
print("Standard deviation of the mean squared errors: ", std(mean_squared_error_list))

Part B
Mean of the mean squared errors:  47.59871490121383
Standard deviation of the mean squared errors:  56.63273442562617


In [ ]:
Observation: MSE improves as we moved to normalised predictors

## Train and Test the Network  (PART C)

In [16]:
#PART C
mean_squared_error_list=[]

# Build the model
model = regression_model()

for i in range (0,50):
    print("Iteration #",i)
    # Train and test the model at the same time using the fit method. 30% of the data for validation.
    history = model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)
    mean_squared_error_list.append(history.history['mean_squared_error'][len(history.history['mean_squared_error'])-1])

Iteration # 0
Train on 721 samples, validate on 309 samples
Epoch 1/100
 - 0s - loss: 1675.8282 - mean_squared_error: 1675.8282 - val_loss: 1199.5353 - val_mean_squared_error: 1199.5353
Epoch 2/100
 - 0s - loss: 1657.8620 - mean_squared_error: 1657.8620 - val_loss: 1185.7325 - val_mean_squared_error: 1185.7325
Epoch 3/100
 - 0s - loss: 1638.9452 - mean_squared_error: 1638.9452 - val_loss: 1171.6907 - val_mean_squared_error: 1171.6907
Epoch 4/100
 - 0s - loss: 1619.2924 - mean_squared_error: 1619.2924 - val_loss: 1156.9361 - val_mean_squared_error: 1156.9361
Epoch 5/100
 - 0s - loss: 1598.5572 - mean_squared_error: 1598.5572 - val_loss: 1141.9073 - val_mean_squared_error: 1141.9073
Epoch 6/100
 - 0s - loss: 1576.7333 - mean_squared_error: 1576.7333 - val_loss: 1126.5011 - val_mean_squared_error: 1126.5011
Epoch 7/100
 - 0s - loss: 1553.5962 - mean_squared_error: 1553.5962 - val_loss: 1110.3026 - val_mean_squared_error: 1110.3026
Epoch 8/100
 - 0s - loss: 1529.2476 - mean_squared_error: 

In [17]:
print("Part C")
print("Mean of the mean squared errors: " , mean(mean_squared_error_list))
print("Standard deviation of the mean squared errors: ", std(mean_squared_error_list))

Part C
Mean of the mean squared errors:  35.78222331722963
Standard deviation of the mean squared errors:  20.524970302536993


In [ ]:
Observation: MSE improves with increase in number of epochs

## Train and Test the Network  (PART D)

In [18]:
#PART D
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
    return model

# Build the model
model = regression_model()
    
mean_squared_error_list=[]

for i in range (0,50):
    print("Iteration #",i)
    # Train and test the model at the same time using the fit method. 30% of the data for validation.
    history = model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2)
    mean_squared_error_list.append(history.history['mean_squared_error'][len(history.history['mean_squared_error'])-1])


Iteration # 0
Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 1671.0208 - mean_squared_error: 1671.0208 - val_loss: 1199.7719 - val_mean_squared_error: 1199.7719
Epoch 2/50
 - 0s - loss: 1644.8913 - mean_squared_error: 1644.8913 - val_loss: 1176.7634 - val_mean_squared_error: 1176.7634
Epoch 3/50
 - 0s - loss: 1603.6548 - mean_squared_error: 1603.6548 - val_loss: 1138.5168 - val_mean_squared_error: 1138.5168
Epoch 4/50
 - 0s - loss: 1529.5094 - mean_squared_error: 1529.5094 - val_loss: 1068.2732 - val_mean_squared_error: 1068.2732
Epoch 5/50
 - 0s - loss: 1393.8985 - mean_squared_error: 1393.8985 - val_loss: 945.7854 - val_mean_squared_error: 945.7854
Epoch 6/50
 - 0s - loss: 1171.9676 - mean_squared_error: 1171.9676 - val_loss: 769.2863 - val_mean_squared_error: 769.2863
Epoch 7/50
 - 0s - loss: 879.8401 - mean_squared_error: 879.8401 - val_loss: 554.0003 - val_mean_squared_error: 554.0003
Epoch 8/50
 - 0s - loss: 578.4963 - mean_squared_error: 578.4963 - val_lo

In [19]:
print("Part D")
print("Mean of the mean squared errors: " , mean(mean_squared_error_list))
print("Standard deviation of the mean squared errors: ", std(mean_squared_error_list))

Part D
Mean of the mean squared errors:  22.84799083412107
Standard deviation of the mean squared errors:  13.98812966919819


In [ ]:
Observation: MSE gets better with addition of more hidden layers